# Корреляция Пирсона

In [1]:
import pandas as pd
import numpy as np

## Foodmart product sales 

In [4]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/4_inference/3_dependencies/data/3.3.1_foodmart.sales.tsv'
sales = pd.read_csv(path, sep = '\t', header = 0, parse_dates = [2])
sales.head()

In [7]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/4_inference/3_dependencies/data/3.3.1_foodmart.products.tsv'
products = pd.read_csv(path, sep = '\t', header = 0)
products.head()

,product_class_id,product_id,brand_name,product_name,SKU,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
0,30,1,Washington,Washington Berry Juice,90748583674,2.85,8.39,6.39,False,False,30,14,16.9,12.60,7.40
1,52,2,Washington,Washington Mango Drink,96516502499,0.74,7.42,4.42,False,True,18,8,13.4,3.71,22.60
2,52,3,Washington,Washington Strawberry Drink,58427771925,0.83,13.10,11.10,True,True,17,13,14.4,11.00,7.77
3,19,4,Washington,Washington Cream Soda,64412155747,3.64,10.60,9.60,True,False,26,10,22.9,18.90,7.93
4,19,5,Washington,Washington Diet Soda,85561191439,2.19,6.66,4.65,True,False,7,10,20.7,21.90,19.20


In [14]:
sales = sales.merge(products[['product_id', 'product_name']], 
                    on = ['product_id'], how = 'inner')
sales.head()

,product_id,store_id,date,sales,product_name_x,product_name_y,product_name
0,4,6,1997-01-01,4,Washington Cream Soda,Washington Cream Soda,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda,Washington Cream Soda,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda,Washington Cream Soda,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda,Washington Cream Soda,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda,Washington Cream Soda,Washington Cream Soda


## Взаимное влияние продуктов друг на друга

Будем оценивать, насколько покупки разных товаров связаны между собой. Для этого рассчитаем корреляцию Пирсона по всем парам продуктов.

In [15]:
sparse_sales = pd.pivot_table(sales, values='sales', index=['date', 'store_id'],
                     columns=['product_name'], fill_value = 0)

In [16]:
sparse_sales.head()

product_name         ADJ Rosy Sunglasses  ...  Washington Strawberry Drink
date       store_id                       ...                             
1997-01-01 6                           0  ...                            0
           14                          0  ...                            0
1997-01-02 11                          0  ...                            0
           23                          0  ...                            0
1997-01-03 7                           0  ...                            0

[5 rows x 1559 columns]

В полученномй таблице каждому продукту соответствуем отдельный столбей.Рассчитываем корреляцию по столбцам.

In [17]:
sales_correlation = sparse_sales.corr()

In [18]:
sales_correlation.head()

product_name,ADJ Rosy Sunglasses,Akron City Map,Akron Eyeglass Screwdriver,American Beef Bologna,American Chicken Hot Dogs,American Cole Slaw,American Corned Beef,American Foot-Long Hot Dogs,American Low Fat Bologna,American Low Fat Cole Slaw,American Pimento Loaf,American Potato Salad,American Roasted Chicken,American Sliced Chicken,American Sliced Ham,American Sliced Turkey,American Turkey Hot Dogs,Amigo Lox,Amigo Scallops,Applause Canned Mixed Fruit,Applause Canned Peaches,Atomic Bubble Gum,Atomic Malted Milk Balls,Atomic Mint Chocolate Bar,Atomic Mints,Atomic Semi-Sweet Chocolate Bar,Atomic Spicy Mints,Atomic Tasty Candy Bar,Atomic White Chocolate Bar,BBB Best Apple Butter,BBB Best Apple Jam,BBB Best Apple Jelly,BBB Best Apple Preserves,BBB Best Brown Sugar,BBB Best Canola Oil,BBB Best Chunky Peanut Butter,BBB Best Columbian Coffee,BBB Best Corn Oil,BBB Best Creamy Peanut Butter,BBB Best Decaf Coffee,...,Tri-State Oranges,Tri-State Party Nuts,Tri-State Peaches,Tri-State Plums,Tri-State Potatos,Tri-State Prepared Salad,Tri-State Red Delcious Apples,Tri-State Red Pepper,Tri-State Shitake Mushrooms,Tri-State Squash,Tri-State Summer Squash,Tri-State Sweet Onion,Tri-State Sweet Peas,Tri-State Tangerines,Tri-State Tomatos,Tri-State Walnuts,Urban Egg Substitute,Urban Large Brown Eggs,Urban Large Eggs,Urban Small Brown Eggs,Urban Small Eggs,Walrus Chablis Wine,Walrus Chardonnay,Walrus Chardonnay Wine,Walrus Imported Beer,Walrus Light Beer,Walrus Light Wine,Walrus Merlot Wine,Walrus White Zinfandel Wine,Washington Apple Drink,Washington Apple Juice,Washington Berry Juice,Washington Cola,Washington Cranberry Juice,Washington Cream Soda,Washington Diet Cola,Washington Diet Soda,Washington Mango Drink,Washington Orange Juice,Washington Strawberry Drink
product_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ADJ Rosy Sunglasses,1.000000,0.076608,-0.006581,0.038685,0.041105,-0.045887,-0.008466,0.062309,0.033752,-0.009656,0.058200,0.038200,0.025900,0.008271,0.030595,0.009877,-0.016423,-0.032879,0.037977,-0.037735,0.074279,0.019257,0.002659,0.029254,0.001567,0.098671,0.029488,0.087733,-0.001663,-0.010218,-0.025424,0.021644,0.023448,-0.007424,-0.046051,-0.031978,0.013124,0.009575,-0.000455,-0.047732,...,0.012305,-0.031720,0.021583,-0.020208,0.006848,0.006705,-0.041629,-0.040474,0.011409,0.030997,0.074147,0.050825,-0.052374,0.085166,-0.000740,0.015713,0.012871,0.037645,-0.034729,-0.007098,0.046148,0.064350,0.025157,0.102532,0.039656,0.013905,0.142755,0.000183,0.032465,-0.046656,0.053533,0.072249,0.061736,-0.000735,0.000843,0.028226,0.012653,0.089932,0.072668,0.035745
Akron City Map,0.076608,1.000000,0.026297,0.088709,0.061144,0.039347,0.019767,-0.005998,0.010270,0.078176,0.033388,-0.010992,0.019015,0.001623,0.080265,0.050194,-0.006202,0.012432,0.073192,0.020603,0.084130,0.033679,0.077270,0.055714,0.020744,0.017377,0.134537,0.119367,-0.038570,0.035823,0.070572,-0.003507,0.077140,0.011547,0.022784,0.086464,0.036002,0.041592,-0.000981,0.043031,...,-0.041506,0.030519,0.029737,0.032795,0.030683,-0.012123,0.060621,0.065596,0.027032,0.002040,0.079388,-0.017654,-0.028525,0.120141,-0.012513,-0.004047,-0.013287,0.060939,0.048864,0.116181,0.027993,0.014841,-0.051571,-0.008078,0.016266,-0.002397,0.014104,0.034586,-0.039916,0.067374,-0.021793,0.020514,0.051956,-0.001018,-0.032582,0.007142,0.035274,0.033875,0.078070,0.051952
Akron Eyeglass Screwdriver,-0.006581,0.026297,1.000000,-0.040851,0.034345,0.108726,0.007573,0.162355,0.020933,0.074292,0.168326,0.022102,-0.002586,0.047054,0.086490,0.025701,0.008815,0.119501,0.062123,-0.030963,0.099969,0.056648,0.128681,0.056870,-0.057635,0.021827,0.020201,0.060103,0.115806,0.057689,0.031874,0.039208,0.031884,0.016561,-0.022387,0.016969,0.049666,0.128798,0.000260,0.036040,...,0.037512,-0.003210,0.027239,0.054571,0.049415,0.095266,0.065042,0.016790,0.031655,0.015059,0.088306,0.063425,0.051336,0.054048,0.004773,0.066565,0.089716,0.093531,0.088077,0.117507,0.042815,0.027568,0.03653

Зафиксируем некоторый продукт и посмотрим, как его покупки коррелируют с покупками других продуктов

In [22]:
product_name = 'American Chicken Hot Dogs'
sales_correlation[[product_name]].sort_values(product_name, ascending=False).head()

product_name,American Chicken Hot Dogs
product_name,
American Chicken Hot Dogs,1.000000
Mighty Good Monthly Home Magazine,0.240269
Big Time Frozen Chicken Wings,0.223398
Carlson Sour Cream,0.220726
Great Cranberry Muffins,0.217509


Посмотрим, насколько велика по модулю может быть отрицательная корреляция между покупками продуктов

In [25]:
min_corr = pd.DataFrame(sales_correlation.min())
min_corr.shape
min_corr.columns = ['min']
min_corr.sort_values(by = 'min').head()

,min
product_name,
Carlson Whole Milk,-0.085064
Pearl Chardonnay Wine,-0.085064
Gulf Coast Malted Milk Balls,-0.082598
Sunset Economy Toilet Brush,-0.082598
Fast Fudge Brownies,-0.082317


Посмотрим, насколько велика по модулю может быть положительная корреляция между покупками продуктов

In [ ]:
max_corr = pd.DataFrame(sales_correlation.apply(lambda x : 
    np.max(list(filter(lambda x : x != 1., x))), axis = 1))
max_corr.columns = ['max']
max_corr.sort_values(by = 'max', ascending = False).head()

,max
product_name,
Plato French Roast Coffee,0.340598
Just Right Vegetable Soup,0.340598
Nationeel Grape Fruit Roll,0.322176
Great Muffins,0.322176
Club Low Fat Cottage Cheese,0.306701


In [ ]:
product_name = 'Plato French Roast Coffee'
sales_correlation[[product_name]].sort_values(product_name, ascending = False).head()

product_name,Plato French Roast Coffee
product_name,
Plato French Roast Coffee,1.000000
Just Right Vegetable Soup,0.340598
Hermanos Macintosh Apples,0.289095
Tell Tale Macintosh Apples,0.273905
Consolidated Whitening Toothpast,0.247477
